In [1]:
from InstructorEmbedding import INSTRUCTOR
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import tiktoken

/home/kpatelis/projects/bggrecommender/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
df = pd.read_csv("data/bgg_gamelist_all_details.csv")

# Drop game with no description
df = df.query('~description.isnull()')

In [4]:
descriptions = list(df["description"])
descriptions[0:5]

['Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a national party membership which will grow as the game progresses and this will supply a fair number of victory points. Lastly, parties score some victory points if their party platform matches the national opinions at the end of the game.&#10;&#10;The 1986 edition featured four parties from the old West Germany and supporte

In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

sum = 0
for description in descriptions:
    sum += num_tokens_from_string(description, "cl100k_base")
    
sum, sum / 1000 * 0.0004

(6756630, 2.702652)

### Try the free alternative - instructor-xl

In [31]:
model = INSTRUCTOR('hkunlp/instructor-xl')
embeddings = []
instruction = "Represent the Game description:"

load INSTRUCTOR_Transformer
max_seq_length  512


In [32]:
for description in tqdm(descriptions):
    embedding = model.encode([[instruction, description]])
    embeddings.append(embedding)

100%|██████████| 24486/24486 [1:19:36<00:00,  5.13it/s]


In [38]:
embs= np.squeeze(np.array(embeddings))

In [42]:
#np.savetxt("bggrecommender/data/embeddings.gz", embs)
np.savez_compressed("bggrecommender/data/embeddings.npz", embeddings = embs)